In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

import pandas as pd
import numpy as np

from collections import Counter
import warnings
warnings.filterwarnings('ignore')

target_cols = ['ind_ahor_fin_ult1','ind_aval_fin_ult1','ind_cco_fin_ult1','ind_cder_fin_ult1','ind_cno_fin_ult1',
               'ind_ctju_fin_ult1','ind_ctma_fin_ult1','ind_ctop_fin_ult1','ind_ctpp_fin_ult1','ind_deco_fin_ult1',
               'ind_deme_fin_ult1','ind_dela_fin_ult1','ind_ecue_fin_ult1','ind_fond_fin_ult1','ind_hip_fin_ult1',
               'ind_plan_fin_ult1','ind_pres_fin_ult1','ind_reca_fin_ult1','ind_tjcr_fin_ult1','ind_valo_fin_ult1',
               'ind_viv_fin_ult1','ind_nomina_ult1','ind_nom_pens_ult1','ind_recibo_ult1']

### Feature Engineering

In [2]:
# read data in
train = pd.read_csv("filtered_train_dataset.csv")
test  = pd.read_csv("../../input/test_ver2.csv")
print(train.info(memory_usage=True))
print(test.info(memory_usage=True))



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2195520 entries, 0 to 2195519
Data columns (total 48 columns):
fecha_dato               object
ncodpers                 int64
ind_empleado             object
pais_residencia          object
sexo                     object
age                      object
fecha_alta               object
ind_nuevo                float64
antiguedad               object
indrel                   float64
ult_fec_cli_1t           object
indrel_1mes              object
tiprel_1mes              object
indresi                  object
indext                   object
conyuemp                 object
canal_entrada            object
indfall                  object
tipodom                  float64
cod_prov                 float64
nomprov                  object
ind_actividad_cliente    float64
renta                    float64
segmento                 object
ind_ahor_fin_ult1        int64
ind_aval_fin_ult1        int64
ind_cco_fin_ult1         int64
ind_cder_fin_ult1    

In [3]:
# it seems that some columns have mixed dtype. We need to unify everything
print(Counter(train.fecha_dato))
print(Counter(test.fecha_dato))

train_rows = train.shape[0]
print(train.shape)
test_rows = test.shape[0]
print(test.shape)

trainX = train.iloc[:,:24]
trainY = train.iloc[:,24:]
print(trainX.shape)

Counter({'2016-05-28': 931453, '2015-06-28': 632110, '2015-05-28': 631957})
Counter({'2016-06-28': 929615})
(2195520, 48)
(929615, 24)
(2195520, 24)


In [4]:
# combine train and test dataset together for preprocessing and cleaning
combined = pd.concat([trainX, test], axis=0)
print("The shape is ", combined.shape)

# the dtype of some columns are not right, correct them, however, you can't convert nan to int in pandas, so use float
combined.age = pd.to_numeric(combined.age, errors='coerce')
combined.renta = pd.to_numeric(combined.renta, errors='coerce')
combined.antiguedad = pd.to_numeric(combined.antiguedad, errors='coerce')
combined.indrel_1mes = pd.to_numeric(combined.indrel_1mes, errors='coerce')

# the 'conyuemp' column is empty at 100000, so remove it
#combined.drop('conyuemp', 1, inplace=True)

# tipodom doesn't seem to be useful, so I am going to drop it
#combined.drop('tipodom', 1, inplace=True)

# As 99% of 'ult_fec_cli_1t' information missing, so I am going to drop it
#combined.drop('ult_fec_cli_1t', 1, inplace=True)

# as we have both cod_prov and nomprov, they represent the same thing. so drop nomprov
combined.drop('nomprov', 1, inplace=True)

# separate numeric from categorical data
combined_num = combined.select_dtypes(exclude=['object'])
combined_cat = combined.select_dtypes(include=['object'])

del combined

The shape is  (3125135, 24)


### Handle Missing Data

In [5]:
### For AGE ###
# for missing age value
combined_num.age.fillna(39, inplace=True)

# replace all the age < 18 to ave of 18-30 and > 90 with ave of 30-90
combined_num.loc[combined_num.age < 18, 'age'] = 23
combined_num.loc[combined_num.age > 90, 'age'] = 50
combined_num.age = combined_num.age.astype(int)

#### for ind_nuevo (New customer)
combined_num['ind_nuevo'].fillna(1.0, inplace=True)

#### for antiguedad (antiquity or Seniority), use the 25 percentile, which is 23
combined_num['antiguedad'].fillna(23, inplace=True)

#### for 'indrel' column, which indicates
# 1 (First/Primary), 99 (Primary customer during the month but not at the end of the month)
# use the most common one which is 1
combined_num['indrel'].fillna(1.0, inplace=True)

#### for 'indrel_1mes' column
# As suggested by @StephenSmith
map_dict = { 1.0  : "1",
            "1.0" : "1",
            "1"   : "1",
            "3.0" : "3",
            "P"   : "5",
            3.0   : "3",
            2.0   : "2",
            "3"   : "3",
            "2.0" : "2",
            4.0   : "4",
            "4"   : "4",
            "2"   : "2"}

combined_num.indrel_1mes.fillna("P",inplace=True)
combined_num.indrel_1mes = combined_num.indrel_1mes.apply(lambda x: map_dict.get(x,x))
combined_num.indrel_1mes = combined_num.indrel_1mes.astype("category")
print(Counter(combined_num.indrel_1mes))

#### for tipodom: (Addres type. 1, primary address), 
# since this one doesn't provide any useful information, so will drop it eventually
combined_num['tipodom'].fillna(1.0, inplace=True)

#### for cod_prov: provincial code for the address, use '0' for the unknown code
combined_num['cod_prov'].fillna(0, inplace=True)

#### ind_actividad_cliente  (Ind_activity customer), use '2.0 for the unknown value
combined_num['ind_actividad_cliente'].fillna(2.0, inplace=True)

#### renta (rental), assign them region by region
rental_dict = {0.0: 190864.28134387353, 1.0: 116454.21824999999, 2.0: 83059.639880275121, 3.0: 87347.808423990966, 
               4.0: 85374.23584159567, 5.0: 76816.518459868807, 6.0: 72176.18582748514, 7.0: 171995.86848114064, 
               8.0: 164672.39357774809, 9.0: 97878.796620592184, 10.0: 75372.170251322314, 11.0: 98648.098741735899, 
               12.0: 79174.239231086744, 13.0: 69896.499221334176, 14.0: 85622.935385531295, 15.0: 112788.86299594035, 
               16.0: 69949.635891681086, 17.0: 144340.73909940803, 18.0: 96553.136598167126, 19.0: 95550.728265982965, 
               20.0: 139632.71175824184, 21.0: 76687.208433018008, 22.0: 89229.944358405774, 23.0: 77132.018859586417, 
               24.0: 93387.127306748385, 25.0: 81230.193312456133, 26.0: 99642.268904048149, 27.0: 76675.52248979923, 
               28.0: 178865.27951727214, 29.0: 121200.83724370257, 30.0: 79075.0041727278, 31.0: 105811.42276119406, 
               32.0: 83299.617043690712, 33.0: 101398.81625954412, 34.0: 92783.771758008908, 35.0: 100323.19958599037, 
               36.0: 113524.32363071061, 37.0: 105792.44947409695, 38.0: 102717.47302208183, 39.0: 121197.46123931887, 
               40.0: 98489.164981292226, 41.0: 117422.48873751337, 42.0: 88050.127969305482, 43.0: 104578.57903186574, 
               44.0: 87686.596356093884, 45.0: 80594.888512939171, 46.0: 89768.963794916985, 47.0: 101652.21498126048, 
               48.0: 110186.07268421052, 49.0: 83348.77014432376, 50.0: 110563.52870700191, 51.0: 199147.44231460703, 
               52.0: 149861.65298934001}

for pcode in rental_dict.keys():
    # fetch rows that are within the pcode and with 'renta' value is np.nan
    idx = combined_num.loc[combined_num.cod_prov.isin([pcode]) & combined_num.renta.isnull(), 'renta'].index.tolist()
    if idx:
        #print(idx)
        combined_num.ix[idx, 'renta'] = rental_dict[pcode]

### For categorical columns ###

### For fecha_alta (joined date)
# Some entries don't have the date they joined the company. Just give them something in the middle of the pack
combined_cat.loc[combined_cat.fecha_alta.isnull(),"fecha_alta"] = '2011-08-31'

#### For conyuemp, assigned an unknown value
combined_cat['conyuemp'].fillna('U', inplace=True)

#### For ind_empleado (employed or employment), I will assign the most common one 'N'
print(Counter(combined_cat.ind_empleado))
combined_cat.loc[combined_cat.ind_empleado.isnull(), 'ind_empleado'] = 'N'

#### For pais_residencia (Country of residency), use the most common one: 'ES'
combined_cat.loc[combined_cat.pais_residencia.isnull(), 'pais_residencia'] = 'ES'

#### For sexo, use unknown category
combined_cat.loc[combined_cat.sexo.isnull(), 'sexo'] = 'U'

#### for ult_fec_cli_1t
combined_cat.loc[combined_cat.ult_fec_cli_1t.isnull(), 'ult_fec_cli_1t'] = '2011-01-11'

#### For indfall, use the most common one
combined_cat.loc[combined_cat.indfall.isnull(), 'indfall'] = 'N'

#### For tiprel_1mes, use an unknown category
combined_cat.loc[combined_cat.tiprel_1mes.isnull(), 'tiprel_1mes'] = 'U'

#### For indresi, use the most common one
combined_cat.loc[combined_cat.indresi.isnull(), 'indresi'] = 'S'

#### For indext, use the most common one
combined_cat.loc[combined_cat.indext.isnull(), 'indext'] = 'N'

#### For canal_entrada (input channel), use an unknown one
combined_cat.loc[combined_cat.canal_entrada.isnull(), 'canal_entrada'] = 'AAA'
print(Counter(combined_cat.canal_entrada))

#### For segmento (segment), use the unknown one
combined_cat.loc[combined_cat.segmento.isnull(), 'segmento'] = '04 - unknown'

### For trainY
#### For ind_nomina_ult1 and ind_nom_pens_ult1, they only in training dataset. 
# Here I will assign them with the most common value, which is '0'
trainY.loc[trainY.ind_nomina_ult1.isnull(), 'ind_nomina_ult1'] = 0
trainY.loc[trainY.ind_nom_pens_ult1.isnull(), 'ind_nom_pens_ult1'] = 0

Counter({'1': 3115079, 'P': 9568, '3': 341, '2': 122, '4': 25})
Counter({'N': 3117675, nan: 5458, 'B': 825, 'F': 589, 'A': 584, 'S': 4})
Counter({'KHE': 927355, 'KAT': 741917, 'KFC': 694797, 'KHQ': 150044, 'KFA': 93653, 'KHM': 66967, 'KHK': 57330, 'KHN': 39470, 'KHD': 26483, 'RED': 19606, 'KAS': 19365, 'KAG': 16903, 'KAY': 14997, 'KAA': 14972, 'AAA': 14288, 'KAB': 14049, 'KAE': 11381, 'KCC': 11158, 'KHL': 10881, 'KBZ': 10426, 'KFD': 9859, 'KAI': 8453, 'KEY': 7900, 'KAW': 7706, 'KAR': 7254, 'KAZ': 7096, 'KAF': 6934, '007': 6323, '013': 6290, 'KCI': 5856, 'KAH': 5585, 'KAJ': 5284, 'KCH': 5268, 'KHF': 4637, 'KAQ': 3980, 'KHC': 3562, 'KAP': 3279, 'KHO': 3272, 'KAM': 2515, 'KAD': 2450, 'KFP': 2101, 'KGX': 2088, 'KEJ': 2006, 'KGV': 1990, 'KDR': 1782, 'KAC': 1749, 'KFT': 1731, 'KAL': 1659, 'KBO': 1620, 'KBH': 1583, 'KFG': 1514, 'KAO': 1466, 'KFS': 1430, 'KFJ': 1424, 'KES': 1342, 'KEW': 1294, 'KCG': 1239, 'KFF': 1216, 'KCB': 1161, 'KEN': 1084, 'KFU': 1062, 'KFN': 991, 'KBQ': 948, 'KCL': 928, '

In [6]:
print("For categorical datan\n", combined_cat.isnull().any())
print("\nFor numeric data:\n", combined_num.isnull().any())
print("\nFor Y training data\n", trainY.isnull().any())
#print("\nshape of the combined data\n", combined.shape)

For categorical datan
 fecha_dato         False
ind_empleado       False
pais_residencia    False
sexo               False
fecha_alta         False
ult_fec_cli_1t     False
tiprel_1mes        False
indresi            False
indext             False
conyuemp           False
canal_entrada      False
indfall            False
segmento           False
dtype: bool

For numeric data:
 ncodpers                 False
age                      False
ind_nuevo                False
antiguedad               False
indrel                   False
indrel_1mes              False
tipodom                  False
cod_prov                 False
ind_actividad_cliente    False
renta                    False
dtype: bool

For Y training data
 ind_ahor_fin_ult1    False
ind_aval_fin_ult1    False
ind_cco_fin_ult1     False
ind_cder_fin_ult1    False
ind_cno_fin_ult1     False
ind_ctju_fin_ult1    False
ind_ctma_fin_ult1    False
ind_ctop_fin_ult1    False
ind_ctpp_fin_ult1    False
ind_deco_fin_ult1    False
ind_de

### Now convert categorical data to numeric

In [7]:
for cat in combined_cat.columns:
    combined_cat[cat], unique = pd.factorize(combined_cat[cat])
    if cat != 'fecha_alta':
        print("for ", cat, "\n", Counter(combined_cat[cat]))

for  fecha_dato 
 Counter({2: 931453, 3: 929615, 1: 632110, 0: 631957})
for  ind_empleado 
 Counter({0: 3123133, 2: 825, 3: 589, 1: 584, 4: 4})
for  pais_residencia 
 Counter({0: 3109793, 8: 1197, 11: 1127, 7: 1085, 9: 1075, 14: 852, 19: 824, 40: 686, 25: 678, 10: 598, 17: 548, 13: 535, 18: 509, 1: 471, 12: 353, 59: 351, 20: 331, 27: 331, 2: 230, 26: 209, 3: 180, 41: 178, 64: 175, 29: 138, 34: 137, 30: 130, 72: 120, 42: 116, 4: 113, 46: 112, 6: 104, 15: 98, 68: 98, 52: 96, 5: 94, 28: 92, 35: 80, 23: 64, 36: 56, 66: 56, 39: 52, 75: 52, 65: 48, 24: 44, 21: 34, 45: 32, 43: 30, 57: 30, 73: 28, 76: 28, 62: 26, 78: 26, 50: 24, 51: 24, 63: 24, 80: 24, 37: 22, 69: 22, 86: 22, 56: 20, 58: 20, 70: 20, 79: 20, 81: 20, 54: 18, 71: 18, 47: 16, 61: 16, 77: 16, 93: 16, 22: 14, 67: 14, 16: 12, 38: 12, 44: 12, 48: 12, 55: 12, 94: 12, 99: 12, 101: 12, 49: 10, 104: 10, 106: 10, 31: 8, 32: 8, 53: 8, 60: 8, 89: 8, 90: 8, 92: 8, 100: 8, 103: 8, 105: 6, 33: 4, 74: 4, 82: 4, 83: 4, 84: 4, 85: 4, 87: 4, 88: 4,

### join them together and then split them up according to training and testing data

In [8]:
joined = pd.concat([combined_num, combined_cat], axis=1)
print("joined shape is: ", joined.shape, " and the type is ", type(joined))
trainX = joined.iloc[:train_rows,:]
testX  = joined.iloc[train_rows:,:]

print("Training X shape: ", trainX.shape)
print("Testing X shape: ", testX.shape)
print("Training Y shape: ", trainY.shape)

# now I need to combine trainX and trainY together
combined = pd.concat([trainX, trainY], axis=1)
print(combined.shape)

joined shape is:  (3125135, 23)  and the type is  <class 'pandas.core.frame.DataFrame'>
Training X shape:  (2195520, 23)
Testing X shape:  (929615, 23)
Training Y shape:  (2195520, 24)
(2195520, 47)


#### Here I am going to find out those people who have added/removed their services
- Counter({'2016-05-28': 931453, '2015-06-28': 632110, '2015-05-28': 631957})
- Counter({2: 931453, 1: 632110, 0: 631957})
  - 0 ==> 2015-05-28;  1 ==> 2015-06-28;  2 ==> 2016-05-28;  3 ==> 2016-06-28
- Counter({'2016-06-28': 929615})

In [9]:
# for 2015
print(Counter(combined.fecha_dato))
train2015_05_28 = combined[combined.fecha_dato.isin([0])]
train2015_05_28_unique = train2015_05_28.copy()
print(train2015_05_28.shape)

train2015_06_28 = combined[combined.fecha_dato.isin([1])]
train2015_06_28_unique = train2015_06_28.copy()
print(train2015_06_28.shape)
print("2015-06-28 unique ones are: ", train2015_06_28_unique.shape)

train2016_05_28 = combined[combined.fecha_dato.isin([2])]
print(train2016_05_28.shape)

print("Test size info is: ", testX[testX.fecha_dato.isin([3])].shape)

# get unique customer IDs
train55customers = set(train2015_05_28['ncodpers'])
train56customers = set(train2015_06_28['ncodpers'])
print("2015-05-28 total customers: ", len(train55customers))
print("2015-06-28 total customers: ", len(train56customers))

# find customers in both 2015-05-28 and 2015-06-28
train_common_customers = train55customers & train56customers
print("Customers are in both 05-28 and 06-28 sets (2015): ", len(train_common_customers))

# new customers only in 2015-06-28
train_unique_in_56 = train56customers - train55customers
print("Customers are only in 06-28 (2015): ", len(train_unique_in_56))

# old customers only in 2015-05-28 who dropped the services
train_unique_in_55 = train55customers - train56customers
print("Customers are only in 05-28 (2015)", len(train_unique_in_55))

train2015_05_28 = train2015_05_28[train2015_05_28.ncodpers.isin(train_common_customers)]
train2015_06_28 = train2015_06_28[train2015_06_28.ncodpers.isin(train_common_customers)]
print(train2015_05_28.shape)
print(train2015_06_28.shape)

# find the unique ones only in 2015-06-28
train2015_06_28_unique = train2015_06_28_unique[train2015_06_28_unique.ncodpers.isin(train_unique_in_56)]
print("2015-06-28 unique ones are (AFTER): ", train2015_06_28_unique.shape)

# find the unique ones only in 2015-05-28
train2015_05_28_unique = train2015_05_28_unique[train2015_05_28_unique.ncodpers.isin(train_unique_in_55)]
print("2015-05-28 unique ones are (AFTER): ", train2015_05_28_unique.shape)

Counter({2: 931453, 1: 632110, 0: 631957})
(631957, 47)
(632110, 47)
2015-06-28 unique ones are:  (632110, 47)
(931453, 47)
Test size info is:  (929615, 23)
2015-05-28 total customers:  631957
2015-06-28 total customers:  632110
Customers are in both 05-28 and 06-28 sets (2015):  628603
Customers are only in 06-28 (2015):  3507
(628603, 47)
(628603, 47)
2015-06-28 unique ones are (AFTER):  (3507, 47)


### Save files to disk for quick access

In [ ]:
#train2015_05_28.to_csv("train2015_05_28.csv", index=False)
#train2015_06_28.to_csv("train2015_06_28.csv", index=False)
#train2015_06_28_unique.to_csv("train2015_06_28_unique.csv", index=False)
#train2015_05_28_unique.to_csv("train2015_05_28_unique.csv", index=False)
#train2016_05_28.to_csv("train2016_05_28.csv", index=False)
#testX.to_csv("testX.csv", index=False)

# for changed ones:
changed_ids = np.load("changed_ids_all.txt")

In [10]:
# need to find out those customers who changed services betweeen 2015-05-28 to 2015-06-28
may_data  = train2015_05_28.sort_values(by='ncodpers').reset_index(drop=True).set_index('ncodpers')[target_cols]
june_data = train2015_06_28.sort_values(by='ncodpers').reset_index(drop=True).set_index('ncodpers')[target_cols]
print(may_data.shape)
print(june_data.shape)
print(june_data[june_data.index==15889])

(628603, 24)
(628603, 24)
          ind_ahor_fin_ult1  ind_aval_fin_ult1  ind_cco_fin_ult1  \
ncodpers                                                           
15889                     0                  0                 1   

          ind_cder_fin_ult1  ind_cno_fin_ult1  ind_ctju_fin_ult1  \
ncodpers                                                           
15889                     0                 0                  0   

          ind_ctma_fin_ult1  ind_ctop_fin_ult1  ind_ctpp_fin_ult1  \
ncodpers                                                            
15889                     0                  0                  1   

          ind_deco_fin_ult1       ...         ind_hip_fin_ult1  \
ncodpers                          ...                            
15889                     0       ...                        0   

          ind_plan_fin_ult1  ind_pres_fin_ult1  ind_reca_fin_ult1  \
ncodpers                                                            
15889              

In [17]:
#differences = (june_data != may_data).any(1)
differences = june_data - may_data
differences[differences < 0] = 0
#print(differences[differences < 0].head(2))

differences["num_new_product"] = differences.sum(axis=1)
differences = differences.loc[differences.num_new_product > 0]
print(differences.shape)
#print(differences.head(2))

accounts_with_new_products = differences.index
print("Accounts have been changed: ", len(accounts_with_new_products))
print("Accounts have been changed ids \n: ", list(accounts_with_new_products)[2:10])

np.save("changed_ids.txt", accounts_with_new_products)

print(len(list(train_common_customers)))

(33318, 25)
Accounts have been changed:  33318
Accounts have been changed ids 
:  [15906, 15925, 15927, 15964, 15982, 16026, 16043, 16056]


TypeError: write() argument must be str, not list

In [ ]:
# need to find out those customers who changed services betweeen 2015-05-28 to 2015-06-28
# to use this approach, I need to make sure their index are the same
from pandas.util.testing import assert_frame_equal

changed_status_55 = []
changed_status_56 = []
train_common_customers = list(train_common_customers)

for id in range(len(train_common_customers)):
    customer = train_common_customers[id]
    #print(customer)
    
    all_2015_05 = may_data[may_data.index == customer]
    all_2015_06 = june_data[june_data.index == customer]
    #print(all_2015_05.shape[0])

    # for services changed
    try:
        assert_frame_equal(all_2015_05, all_2015_06)
    except:
        #changed_status_55 = pd.concat([changed_status_55, all_2015_05])
        #changed_status_56 = pd.concat([changed_status_56, all_2015_06])
        changed_status_55.append(customer)
        changed_status_56.append(customer)
        
print("2015-05 total remaining: ", len(changed_status_55))
print("2015-06 total remaining: ", len(changed_status_56))

In [142]:
#print(train2015_05_28.iloc[:,24:].head(2))
#print(train2015_05_28.head())
print(train2015_05_28.loc[train2015_05_28.ncodpers == 1048576].iloc[:,24:])
print(train2015_05_28.loc[train2015_05_28.ncodpers == 1061283].shape)
#print(train2015_05_28.head().iloc[:,24:])

       ind_aval_fin_ult1  ind_cco_fin_ult1  ind_cder_fin_ult1  \
34226                  0                 1                  0   

       ind_cno_fin_ult1  ind_ctju_fin_ult1  ind_ctma_fin_ult1  \
34226                 0                  0                  0   

       ind_ctop_fin_ult1  ind_ctpp_fin_ult1  ind_deco_fin_ult1  \
34226                  0                  0                  0   

       ind_deme_fin_ult1       ...         ind_hip_fin_ult1  \
34226                  0       ...                        0   

       ind_plan_fin_ult1  ind_pres_fin_ult1  ind_reca_fin_ult1  \
34226                  0                  0                  0   

       ind_tjcr_fin_ult1  ind_valo_fin_ult1  ind_viv_fin_ult1  \
34226                  0                  0                 0   

       ind_nomina_ult1  ind_nom_pens_ult1  ind_recibo_ult1  
34226              0.0                0.0                0  

[1 rows x 23 columns]
(1, 47)
